In [Quick Start](https://github.com/DataCanvasIO/HyperTS/blob/main/examples/02_quick_start.ipynb), we learned the basics of HyperTS modeling:

In [ ]:
from hyperts import make_experiment
from hyperts.datasets import load_network_traffic
from sklearn.model_selection import train_test_split

df = load_network_traffic()
train_data, test_data = train_test_split(df, test_size=168, shuffle=False)

experiment = make_experiment(train_data,
                            task='forecast',
                            timestamp='TimeStamp',
                            covariables=['HourSin', 'WeekCos', 'CBWD'])
model = experiment.run()

X_test, y_test = model.split_X_y(test_data)
forecast = model.predict(X_test)
scores = model.evaluate(y_true=y_test, y_pred=forecast)

To get a better understanding of HyperTS, this NoteBook will go through the ```make_experiment``` tutorial in more detail, so you can explore more robust performance.

1. Create an experiment by default.
2. Select operation mode.
3. Specify the evaluation metrics.
4. Specify optimization direction.
5. Set the maximum search trials.
6. Set early stoping.
7. Specify the validation data set.
8. Specify search algorithm.
9. Specify time frequency.
10. Specify forecast window.
11. Fixed random seed.
12. Adjusting a log level.
13. Discrete time series forecasting.
14. Forecasting without timestamp column.
15. Forecasting train data cut off.
16. Set cross validation.
17. Ensemble models.

#### 1. Create an experiment by default.

First, we must tell the experiment what type of task we are going to do, that is, assign a value to the parameter ```task```. 

Second, in the forecasting task, we must pass the column name of the parameter ```timestamp``` to ```make_experiment```. 
If there are covariates, we also need to pass in the column names of ```covariables```(or ``covariates``). 

Thrid, in the classification task, if the target column of the data is not y or target, the parameter ```target``` needs to be set.

In [ ]:
# Forecast
experiment = make_experiment(train_data, 
                             task='forecast',
                             timestamp='TimeStamp',
                             covariates=['HourSin', 'WeekCos', 'CBWD'])
# Classification
experiment = make_experiment(train_data, task='classification', target='y')  

# Regression
experiment = make_experiment(train_data, task='regression', target='y')  

**Note**

For the time series forecasting task, it may be divided into *univariate forecasting* and *multivariate forecasting* according to the number of the predicted variables. For the time series classification task, data can be divided into *univariate binary classification*, *univariate multiclassification*, *multivariate binary classification* and *multivariate multiclassification* according to the number and classes of feature nad target variables. If we already know the basic situation of the data and the task to be solved after getting the data, it is recommended to pass the following parameters in the configuration task: 

- multivariate forecasting: task='unvariate-forecast';
- multivariate forecasting: task='multivariate-forecast';
- univariate binary classification: task='univariate-binaryclass';
- univariate multiclassification: task='univariate-multiclass';
- multivariate binary classification: task='multivariate-binaryclass';:
- multivariate multiclassification: task='multivariate-multiclass'.

Of course, we can also simply configure ```task='forecast'```, ```task='classification'``` and ```task='regression'```. So that, HyperTS will perform detailed task type inference from the data combined with other known column information.

--------------

#### 2. Select operation mode.

HyperTS has three built-in modeling modes, namely statistical model mode ('stats'), deep learning mode ('dl') and neural architecture search mode ('nas', not open). By default, the statistical model mode is selected by default, but you can change to other modes as well:

In [ ]:
experiment = make_experiment(train_data, 
                             mode='dl',
                             ...)

The deep learning mode based on Tensorflow, can support GPU. By default, the default experiment will run in the CPU environment. If your device supports GPU and installed the gpu version of tensorflow-gpu, you can change the parameter ```tf_gpu_usage_strategy```:

In [ ]:
experiment = make_experiment(train_data, 
                             mode='dl',
                             tf_gpu_usage_strategy=1,
                             ...)  

where, ```tf_gpu_usage_strategy``` supports three configuration strategies:

- 0: CPU;
- 1: GPU memory growth;
- 2: GPU memory limit, the default is 2048M, and the parameter ```tf_memory_limit``` supports custom configuration.

#### 3. Specify the evaluation metrics.

By default, the model evaluation metric is 'mae' for prediction tasks, 'accuracy' for classification tasks, and 'rmse' for regression tasks. We can reset the evaluation metric through the parameter ```reward_metric```, which can be 'str' or a built-in function of ```sklearn.metrics```, for example:

In [ ]:
# str
experiment = make_experiment(train_data, 
                             task='univariate-binaryclass',
                             reward_metric='auc',
                             ...)  

# sklearn.metrics
from sklearn.metrics import auc
experiment = make_experiment(train_data, 
                             task='univariate-binaryclass',
                             reward_metric=auc,
                             ...) 

Currently, ```reward_metric``` can support a variety of evaluation metrics, as follows:

- Classification: accuracy, auc, f1, precision, recall, logloss.
- Forecasting and regression: mae, mse, rmse, mape, smape, msle, r2.

 For custom ```reward_metric```, please refer to [06_custom_reward_metric.ipynb](https://github.com/DataCanvasIO/HyperTS/blob/main/examples/06_custom_reward_metric.ipynb).

------------------

#### 4. Specify optimization direction.

In the model search phase, the searcher needs to specify the search direction, which will be detected from reward_metric by default. We can also specify via the parameter ```optimize_direction``` ('min' or 'max'):

In [ ]:
experiment = make_experiment(train_data, 
                             task='univariate-binaryclass',
                             reward_metric='auc',
                             optimize_direction='max',
                             ...)  

<br>

#### 5. Set the maximum search trials.

By default, the experiment searchs for 3 parameter models and stops searching. In practice, it is recommended to set the parameter ```max_trials``` to more than 30. If time is sufficient, a larger number of searches will have a higher chance of obtaining a better model:

In [ ]:
experiment = make_experiment(train_data, 
                             max_trials=100,
                             ...) 

<br>

#### 6. Set early stoping.

When the ```max_trials``` setting is large, it may take more time to wait for the experiment to finish. In order to control the rhythm of the work, you can control it through the **Early Stopping** mechanism:

In [ ]:
experiment = make_experiment(train_data, 
                             max_trials=100,
                             early_stopping_time_limit=3600 * 3,  # 3 hours
                             ...)    

where, ```make_experiment``` contains three early stop mechanisms, which can be used together. Details as follows:

- early_stopping_time_limit: Limits the running time of the experiment, with a granularity of seconds.
- early_stopping_round: Limit the number of search rounds for the experiment, with a granularity of times.
- early_stopping_reward: Specify a limit for reward points.

---------------

#### 7. Specify the validation data set.

The experiment object requires not only the training data set, but also the evaluation data set. By default, a part of the evaluation data set will be divided from the training data set in a certain proportion. You can also specify the evaluation set through eval_data during ```make_experiment```, such as:

In [ ]:
experiment = make_experiment(train_data, 
                             eval_data=eval_data,
                             ...) 

Of course, we can also specify the size of the evaluation dataset by setting ```eval_size```:

In [ ]:
experiment = make_experiment(train_data, 
                             eval_size=0.3,
                             ...) 

<br>

#### 8. Specify search algorithm.

HyperTS performs model selection and hyperparameter optimization through the built-in search algorithms in [Hypernets](https://github.com/DataCanvasIO/Hypernets), including EvolutionSearcher (default, 'evolution'), MCTSSearcher ('mcts'), and RandomSearch ('random'), etc. It can be specified by the parameter ```searcher```, specifying the class name of the search algorithm (class) or the name of the search algorithm (str):

In [ ]:
experiment = make_experiment(train_data, 
                             searcher='random',
                             ...)  

In [ ]:
from hypernets.searchers import EvolutionSearcher

search_space_general = ...

experiment = make_experiment(train_data, 
                             searcher=EvolutionSearcher(search_space_general, population_size=500, sample_size=20, candidates_size=20),
                             ...)  

For a detailed description of various search algorithms, please refer to [Search Algorithms](https://hypernets.readthedocs.io/en/latest/searchers.html).

#### 9. Specify time frequency.

In time series forecasting tasks, if we know the temporal frequency of the dataset, you can specify it precisely with the parameter ```freq```:

In [ ]:
experiment = make_experiment(train_data, 
                             task='forecast',
                             timestamp='TimeStamp',
                             freq='H',
                             ...) 

By default, frequency will be inferred from ```timestamp```.

#### 10. Specify forecast window.

When using the deep learning mode for time series forecasting, we can specify the size of the sliding window through the parameter ```forecast_window``` after analyzing the actual situation of the data based on experience:

In [ ]:
experiment = make_experiment(train_data, 
                             task='forecast',
                             mode='dl',
                             timestamp='TimeStamp',
                             dl_forecast_window=24*7,
                             ...)

<br>

#### 11. Fixed random seed.

Sometimes in order to ensure that the experimental results can be reproduced, we need to keep the same initialization. In this case, we can fix the random seed through the parameter ```random_state```:

In [ ]:
experiment = make_experiment(train_data, 
                             random_state=0,
                             ...)  

#### 12. Adjusting a log level.

The progress messages during training can be printed by setting ```log_level``` (str or int). Please refer the logging package of python for further details. Besides, more comprehensive messages will be printed when setting ```verbose``` as 1.

In [ ]:
experiment = make_experiment(train_data, 
                             log_level='INFO', 
                             verbose=1,
                             ...)  

#### 13. Discrete time series forecasting.

In some time series forecasting tasks, there may be no regular time frequency, i.e., discontinuous sampling. At this point, users can set ``mode='dl'`` and ``freq='null'`` to run experiment.

In [ ]:
experiment = make_experiment(train_data,
                            task='forecast',
                            timestamp='TimeStamp',
                            freq='null',
                            ...)

#### 14. Forecasting without timestamp column.

For some time series forecasting data, there might be timestamp column, that is, only the target columns and covariates are contained. In this case, users could set ``timestamp='null'`` to run experiment.

In [ ]:
experiment = make_experiment(train_data,
                            task='forecast',
                            timestamp='null',
                            ...)

In addition, if the sampling frequency of data is known, it is recommeded to specify it by parameter ``freq``, which will facilitate data processing.

#### 15. Forecasting train data cut off.

In the time series forecasting task, if the early too long historical data is involved in the training of the model, it may affect the final performance due to concept drift. ``forecast_train_data_periods`` can cut off the data for the specified period from the end of the training data forward.

In [ ]:
experiment = make_experiment(train_data,
                            task='forecast',
                            mode='stats',
                            timestamp='TimeStamp',
                            forecast_train_data_periods=24*10,
                            ...)

#### 16. Set cross validation.

To enhance the robustness of the model, users can specify whether to enable cross-validation through the parameter ``cv``. When ``cv`` is set to ``True``, it means that cross-validation is enabled, and the number of folds can be set by the parameter ``num_folds`` (default: 3).

In [ ]:
experiment = make_experiment(train_data,
                            cv==True,
                            num_folds=5,
                            ...)

#### 17. Ensemble models.

In order to obtain better model performace, ``make_experiment`` can enable the model ensemble feature when creating an experiment, that is, specify the number of optimal models participating in the ensemble through the parameter ``ensemble_size``. When ``ensemble_size`` is set to ``None`` then model fusion is disabled (default).

In [ ]:
experiment = make_experiment(train_data,
                            ensemble_size=10,
                            max_trials=100,
                            ...)